In [2]:
class SudokuCSP:
    """Sudoku as a Constraint Satisfaction Problem"""
    
    def __init__(self, puzzle):
        """
        Initialize Sudoku CSP.
        puzzle: 9x9 list where 0 represents empty cell
        """
        self.size = 9
        self.box_size = 3
        self.puzzle = [row[:] for row in puzzle]  # Deep copy
        
        # Variables: each cell (i,j)
        # Domains: possible values for each cell
        self.domains = {}
        for i in range(self.size):
            for j in range(self.size):
                if puzzle[i][j] == 0:
                    # Empty cell: domain is all digits 1-9
                    self.domains[(i, j)] = set(range(1, 10))
                else:
                    # Given cell: domain is fixed value
                    self.domains[(i, j)] = {puzzle[i][j]}
    
    def get_row_neighbors(self, row, col):
        """Get all cells in the same row"""
        return [(row, c) for c in range(self.size) if c != col]
    
    def get_col_neighbors(self, row, col):
        """Get all cells in the same column"""
        return [(r, col) for r in range(self.size) if r != row]
    
    def get_box_neighbors(self, row, col):
        """Get all cells in the same 3x3 box"""
        box_row = (row // self.box_size) * self.box_size
        box_col = (col // self.box_size) * self.box_size
        neighbors = []
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if (i, j) != (row, col):
                    neighbors.append((i, j))
        return neighbors
    
    def get_all_neighbors(self, row, col):
        """Get all cells that constrain this cell"""
        neighbors = set()
        neighbors.update(self.get_row_neighbors(row, col))
        neighbors.update(self.get_col_neighbors(row, col))
        neighbors.update(self.get_box_neighbors(row, col))
        return list(neighbors)
    
    def is_consistent(self, row, col, value):
        """
        Check if assigning value to (row, col) violates constraints.
        A value is consistent if it doesn't appear in any neighbor.
        """
        # Check row constraint
        for c in range(self.size):
            if c != col and self.puzzle[row][c] == value:
                return False
        
        # Check column constraint
        for r in range(self.size):
            if r != row and self.puzzle[r][col] == value:
                return False
        
        # Check box constraint
        box_row = (row // self.box_size) * self.box_size
        box_col = (col // self.box_size) * self.box_size
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if (i, j) != (row, col) and self.puzzle[i][j] == value:
                    return False
        
        return True
    
    def is_complete(self):
        """Check if all cells are assigned"""
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle[i][j] == 0:
                    return False
        return True
    
    def print_puzzle(self, title="Sudoku"):
        """Pretty print the puzzle"""
        print(f"\n{title}")
        print("─" * 25)
        for i in range(self.size):
            if i > 0 and i % 3 == 0:
                print("─" * 25)
            row_str = ""
            for j in range(self.size):
                if j > 0 and j % 3 == 0:
                    row_str += "│ "
                val = self.puzzle[i][j]
                row_str += str(val) if val != 0 else "."
                row_str += " "
            print(row_str)
        print()

# Demonstration: Easy 4x4 Sudoku (for clarity)
print("=== CSP Formulation of Sudoku ===\n")

# Simple 4x4 Sudoku for demonstration (2x2 boxes)
# This makes it easier to see all variables and constraints
simple_puzzle_4x4 = [
    [1, 0, 0, 4],
    [0, 4, 1, 0],
    [4, 0, 0, 1],
    [0, 1, 4, 0]
]

print("4x4 Sudoku (2x2 boxes):")
print("Variables: 16 cells")
print("Domains: {1, 2, 3, 4} for each cell")
print("Constraints: No repeats in rows, columns, or 2x2 boxes\n")

# Show the puzzle structure
for i, row in enumerate(simple_puzzle_4x4):
    print(f"Row {i}: {row}")

print("\n--- Constraint Analysis ---\n")

# Demonstrate constraint checking
test_cell = (0, 1)  # Second cell in first row
print(f"Cell {test_cell}: Currently empty (0)")
print(f"Row neighbors: {[(0, c) for c in range(4) if c != test_cell[1]]}")
print(f"Column neighbors: {[(r, 1) for r in range(4) if r != test_cell[0]]}")
print(f"Box neighbors: {[(r, c) for r in range(2) for c in range(2) if (r,c) != test_cell]}")

print(f"\nCurrent row values: {simple_puzzle_4x4[0]}")
print(f"Values 1 and 4 already used in row → cannot assign 1 or 4")
print(f"Therefore, domain for cell {test_cell} = {{2, 3}}")

# Standard 9x9 Sudoku (easy puzzle)
easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Easy Sudoku Puzzle")

# Show domain information
print("Domain Analysis:")
empty_cells = [(i, j) for i in range(9) for j in range(9) if easy_puzzle[i][j] == 0]
print(f"Total variables: 81")
print(f"Given values: {81 - len(empty_cells)}")
print(f"Empty cells (variables to solve): {len(empty_cells)}")
print(f"\nSample domains (first 5 empty cells):")
for cell in empty_cells[:5]:
    print(f"  Cell {cell}: domain = {sorted(csp.domains[cell])}")

=== CSP Formulation of Sudoku ===

4x4 Sudoku (2x2 boxes):
Variables: 16 cells
Domains: {1, 2, 3, 4} for each cell
Constraints: No repeats in rows, columns, or 2x2 boxes

Row 0: [1, 0, 0, 4]
Row 1: [0, 4, 1, 0]
Row 2: [4, 0, 0, 1]
Row 3: [0, 1, 4, 0]

--- Constraint Analysis ---

Cell (0, 1): Currently empty (0)
Row neighbors: [(0, 0), (0, 2), (0, 3)]
Column neighbors: [(1, 1), (2, 1), (3, 1)]
Box neighbors: [(0, 0), (1, 0), (1, 1)]

Current row values: [1, 0, 0, 4]
Values 1 and 4 already used in row → cannot assign 1 or 4
Therefore, domain for cell (0, 1) = {2, 3}

Easy Sudoku Puzzle
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 

Domain Analysis:
Total variables: 81
Given values: 30
Empty cells (variables to solve): 51

Sample domains (first 5 empty 

In [3]:
import time

class NaiveBacktrackingSolver:
    """Basic backtracking without any optimizations"""
    
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0  # Counter for performance analysis
        self.backtracks = 0
    
    def solve(self):
        """Find solution using naive backtracking"""
        self.assignments = 0
        self.backtracks = 0
        start_time = time.time()
        
        result = self._backtrack()
        
        elapsed = time.time() - start_time
        return result, elapsed
    
    def _backtrack(self):
        """Recursive backtracking"""
        # Check if assignment is complete
        if self.csp.is_complete():
            return True
        
        # Select first unassigned variable (naive ordering)
        var = self._select_unassigned_variable()
        if var is None:
            return True
        
        row, col = var
        
        # Try each value in domain (naive ordering: 1-9)
        for value in range(1, 10):
            self.assignments += 1
            
            # Check if value is consistent with current assignment
            if self.csp.is_consistent(row, col, value):
                # Assign value
                self.csp.puzzle[row][col] = value
                
                # Recursive call
                result = self._backtrack()
                if result:
                    return True
                
                # Undo assignment (backtrack)
                self.csp.puzzle[row][col] = 0
                self.backtracks += 1
        
        return False
    
    def _select_unassigned_variable(self):
        """Select first empty cell (naive, left-to-right, top-to-bottom)"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    return (i, j)
        return None

# Test on easy puzzle
print("=== Naive Backtracking Search ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Initial Puzzle")

solver = NaiveBacktrackingSolver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved Puzzle")
    print(f"Success!")
    print(f"Assignments tried: {solver.assignments}")
    print(f"Backtracks: {solver.backtracks}")
    print(f"Time: {elapsed:.4f} seconds")
else:
    print("No solution found")

print("\n--- Performance Analysis ---")
print(f"Search tree nodes explored: {solver.assignments}")
print(f"Wasted effort (backtracks): {solver.backtracks}")
print(f"Efficiency ratio: {solver.backtracks / max(1, solver.assignments):.2%} wasted")

# Compare with slightly harder puzzle
print("\n=== Testing on Medium Puzzle ===\n")

medium_puzzle = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

csp2 = SudokuCSP(medium_puzzle)
solver2 = NaiveBacktrackingSolver(csp2)
solved2, elapsed2 = solver2.solve()

if solved2:
    csp.print_puzzle("Solved Puzzle")
    print(f"Success!")
    print(f"Assignments tried: {solver2.assignments}")
    print(f"Backtracks: {solver2.backtracks}")
    print(f"Time: {elapsed2:.4f} seconds")
else:
    print("No solution found")

print("\n--- Performance Analysis ---")
print(f"Search tree nodes explored: {solver2.assignments}")
print(f"Wasted effort (backtracks): {solver2.backtracks}")
print(f"Efficiency ratio: {solver2.backtracks / max(1, solver2.assignments):.2%} wasted")

=== Naive Backtracking Search ===


Initial Puzzle
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 


Solved Puzzle
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 

Success!
Assignments tried: 37652
Backtracks: 4157
Time: 0.0394 seconds

--- Performance Analysis ---
Search tree nodes explored: 37652
Wasted effort (backtracks): 4157
Efficiency ratio: 11.04% wasted

=== Testing on Medium Puzzle ===


Solved Puzzle
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
────────────────

In [4]:
from copy import deepcopy

class ForwardCheckingSolver:
    """Backtracking with forward checking"""
    
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0
        self.backtracks = 0
        self.domain_reductions = 0
    
    def solve(self):
        """Solve using forward checking"""
        self.assignments = 0
        self.backtracks = 0
        self.domain_reductions = 0
        start_time = time.time()
        
        # Initialize domains by removing values inconsistent with givens
        self._initial_forward_check()
        
        result = self._backtrack()
        elapsed = time.time() - start_time
        return result, elapsed
    
    def _initial_forward_check(self):
        """Remove values from domains based on initial assignments"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] != 0:
                    value = self.csp.puzzle[i][j]
                    # Remove this value from all neighbors
                    for neighbor in self.csp.get_all_neighbors(i, j):
                        ni, nj = neighbor
                        if value in self.csp.domains[(ni, nj)]:
                            self.csp.domains[(ni, nj)].discard(value)
    
    def _backtrack(self):
        """Recursive backtracking with forward checking"""
        if self.csp.is_complete():
            return True
        
        # Select unassigned variable
        var = self._select_unassigned_variable()
        if var is None:
            return True
        
        row, col = var
        
        # Try each value in this variable's domain
        for value in list(self.csp.domains[(row, col)]):
            self.assignments += 1
            
            if self.csp.is_consistent(row, col, value):
                # Assign value
                self.csp.puzzle[row][col] = value
                
                # Save domains before forward checking
                saved_domains = deepcopy(self.csp.domains)
                
                # Forward check: remove value from neighbor domains
                if self._forward_check(row, col, value):
                    # No domain wipeout, continue search
                    result = self._backtrack()
                    if result:
                        return True
                
                # Restore domains and undo assignment
                self.csp.domains = saved_domains
                self.csp.puzzle[row][col] = 0
                self.backtracks += 1
        
        return False
    
    def _forward_check(self, row, col, value):
        """
        Remove value from all neighbor domains.
        Returns False if any domain becomes empty (wipeout).
        """
        for neighbor in self.csp.get_all_neighbors(row, col):
            ni, nj = neighbor
            if self.csp.puzzle[ni][nj] == 0:  # Unassigned
                if value in self.csp.domains[(ni, nj)]:
                    self.csp.domains[(ni, nj)].discard(value)
                    self.domain_reductions += 1
                    
                    # Check for domain wipeout
                    if len(self.csp.domains[(ni, nj)]) == 0:
                        return False
        
        return True
    
    def _select_unassigned_variable(self):
        """Select first unassigned variable (naive ordering)"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    return (i, j)
        return None

# Test forward checking
print("=== Forward Checking ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
print("Domains after initialization:")
empty_cells = [(i, j) for i in range(9) for j in range(9) if easy_puzzle[i][j] == 0]
print(f"Sample cell (0,2): domain = {sorted(csp.domains[(0, 2)])}")

solver = ForwardCheckingSolver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved with Forward Checking")
    print(f"Assignments: {solver.assignments}")
    print(f"Backtracks: {solver.backtracks}")
    print(f"Domain reductions: {solver.domain_reductions}")
    print(f"Time: {elapsed:.4f} seconds")

# Compare with naive backtracking
print("\n--- Comparison: Forward Checking vs Naive Backtracking ---\n")

csp_naive = SudokuCSP(easy_puzzle)
solver_naive = NaiveBacktrackingSolver(csp_naive)
_, time_naive = solver_naive.solve()

print(f"{'Metric':<25} {'Naive':<15} {'Forward Checking':<15} {'Improvement'}")
print("─" * 70)
print(f"{'Assignments':<25} {solver_naive.assignments:<15} {solver.assignments:<15} {solver_naive.assignments/max(1,solver.assignments):.1f}x fewer")
print(f"{'Time (seconds)':<25} {time_naive:<15.4f} {elapsed:<15.4f} {time_naive/max(0.0001,elapsed):.1f}x faster")
print(f"{'Backtracks':<25} {solver_naive.backtracks:<15} {solver.backtracks:<15} {solver_naive.backtracks/max(1,solver.backtracks):.1f}x fewer")

=== Forward Checking ===

Domains after initialization:
Sample cell (0,2): domain = [1, 2, 3, 4, 5, 6, 7, 8, 9]

Solved with Forward Checking
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 

Assignments: 490
Backtracks: 439
Domain reductions: 1480
Time: 0.1328 seconds

--- Comparison: Forward Checking vs Naive Backtracking ---

Metric                    Naive           Forward Checking Improvement
──────────────────────────────────────────────────────────────────────
Assignments               37652           490             76.8x fewer
Time (seconds)            0.0432          0.1328          0.3x faster
Backtracks                4157            439             9.5x fewer


In [5]:
from collections import deque
from copy import deepcopy
import time

class AC3Solver:
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0
        self.backtracks = 0
        self.arc_revisions = 0
        self.propagation_rounds = 0

    def solve(self):
        self.assignments = 0
        self.backtracks = 0
        self.arc_revisions = 0
        self.propagation_rounds = 0

        start_time = time.time()

        # Initial AC-3 preprocessing
        if not self.ac3():
            return False, time.time() - start_time

        # If domains are all singletons, fill puzzle
        if self._all_domains_singleton():
            self._commit_domains_to_puzzle()
            return True, time.time() - start_time

        # Otherwise, search
        result = self._backtrack()
        return result, time.time() - start_time

    # -----------------------------------------------------
    # AC-3 Core
    # -----------------------------------------------------

    def ac3(self, arcs=None):
        """
        Enforce arc consistency.
        Returns False if any domain becomes empty.
        """
        if arcs is None:
            queue = deque()

            # Add all arcs (Xi, Xj) where Xi and Xj are neighbors
            for Xi in self.csp.domains:
                for Xj in self.csp.get_all_neighbors(*Xi):
                    queue.append((Xi, Xj))
        else:
            queue = deque(arcs)

        while queue:
            self.propagation_rounds += 1
            Xi, Xj = queue.popleft()

            if self._revise(Xi, Xj):
                if len(self.csp.domains[Xi]) == 0:
                    return False  # Inconsistency detected

                # If domain changed, re-add all neighbors except Xj
                for Xk in self.csp.get_all_neighbors(*Xi):
                    if Xk != Xj:
                        queue.append((Xk, Xi))

        return True

    def _revise(self, Xi, Xj):
        """
        Make Xi arc-consistent with Xj.
        Remove values from Xi's domain that have no support in Xj.
        """
        revised = False
        self.arc_revisions += 1

        values_to_remove = set()

        for x in self.csp.domains[Xi]:
            # Check if there exists some y in Xj domain such that x != y
            if not any(x != y for y in self.csp.domains[Xj]):
                values_to_remove.add(x)

        if values_to_remove:
            self.csp.domains[Xi] -= values_to_remove
            revised = True

        return revised

    # -----------------------------------------------------
    # Backtracking Search with AC-3 Propagation
    # -----------------------------------------------------

    def _backtrack(self):
        if self._all_domains_singleton():
            self._commit_domains_to_puzzle()
            return True

        var = self._select_unassigned_variable()
        if var is None:
            return True

        for value in list(self.csp.domains[var]):
            self.assignments += 1

            # Save state
            saved_domains = deepcopy(self.csp.domains)

            # Assign by reducing domain to singleton
            self.csp.domains[var] = {value}

            # Run AC-3 with affected arcs only
            arcs = [(neighbor, var)
                    for neighbor in self.csp.get_all_neighbors(*var)]

            if self.ac3(arcs):
                result = self._backtrack()
                if result:
                    return True

            # Restore state
            self.csp.domains = saved_domains
            self.backtracks += 1

        return False

    # -----------------------------------------------------
    # Helpers
    # -----------------------------------------------------

    def _select_unassigned_variable(self):
        """
        Simple variable selection:
        choose variable with domain size > 1.
        """
        for var, domain in self.csp.domains.items():
            if len(domain) > 1:
                return var
        return None

    def _all_domains_singleton(self):
        return all(len(domain) == 1 for domain in self.csp.domains.values())

    def _commit_domains_to_puzzle(self):
        """
        After solving, copy domains into puzzle grid.
        """
        for (i, j), domain in self.csp.domains.items():
            self.csp.puzzle[i][j] = next(iter(domain))


# Test AC-3
print("=== AC-3 Constraint Propagation ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Initial Puzzle")

# Show domains before AC-3
print("Sample domains before AC-3:")
empty_before = [(i, j) for i in range(9) for j in range(9) if easy_puzzle[i][j] == 0]
for cell in empty_before[:3]:
    print(f"  Cell {cell}: {sorted(csp.domains[cell])}")

solver = AC3Solver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved with AC-3")
    print(f"Arc revisions: {solver.arc_revisions}")
    print(f"Assignments needed: {solver.assignments}")
    print(f"Time: {elapsed:.4f} seconds")

# Show the power of AC-3
print("\n--- AC-3 Performance Analysis ---")
print(f"Propagation rounds: {solver.propagation_rounds}")
print(f"Search assignments: {solver.assignments}")
if solver.assignments == 0:
    print("✓ Puzzle solved by pure constraint propagation!")
else:
    print(f"Remaining search needed: {solver.assignments} assignments")

=== AC-3 Constraint Propagation ===


Initial Puzzle
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 

Sample domains before AC-3:
  Cell (0, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9]
  Cell (0, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9]
  Cell (0, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9]

Solved with AC-3
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 

Arc revisions: 9372
Assignments needed: 0
Time: 0.0143 seconds

--- AC-3 Performance Analysis ---
Propagation rounds: 9372
Search assignments: 0
✓ Puzzle solved by pure constraint propagation!


In [6]:
class HeuristicSolver:
    """Backtracking with MRV and LCV heuristics"""
    
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0
        self.backtracks = 0
        self.mrv_selections = []
    
    def solve(self):
        """Solve using intelligent heuristics"""
        self.assignments = 0
        self.backtracks = 0
        self.mrv_selections = []
        
        start_time = time.time()
        
        # Initial AC-3
        ac3_solver = AC3Solver(self.csp)
        if not ac3_solver.ac3():
            return False, 0
        
        result = self._backtrack()
        elapsed = time.time() - start_time
        return result, elapsed
    
    def _select_unassigned_variable_mrv(self):
        """
        Select variable with Minimum Remaining Values.
        Ties broken by degree heuristic.
        """
        min_domain_size = float('inf')
        best_vars = []
        
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    domain_size = len(self.csp.domains[(i, j)])
                    
                    if domain_size < min_domain_size:
                        min_domain_size = domain_size
                        best_vars = [(i, j)]
                    elif domain_size == min_domain_size:
                        best_vars.append((i, j))
        
        if not best_vars:
            return None
        
        # Tie-breaker: degree heuristic (most constraining variable)
        if len(best_vars) > 1:
            max_degree = -1
            best_var = best_vars[0]
            for var in best_vars:
                # Count unassigned neighbors
                degree = sum(1 for n in self.csp.get_all_neighbors(*var) 
                           if self.csp.puzzle[n[0]][n[1]] == 0)
                if degree > max_degree:
                    max_degree = degree
                    best_var = var
            return best_var
        
        self.mrv_selections.append((best_vars[0], min_domain_size))
        return best_vars[0]
    
    def _order_domain_values(self, var):
        """
        Order values using Least Constraining Value heuristic.
        Prefer values that rule out fewer choices for neighbors.
        """
        row, col = var
        domain = list(self.csp.domains[var])
        
        # Count how many neighbor values each value would eliminate
        value_constraints = []
        for value in domain:
            eliminated_count = 0
            for neighbor in self.csp.get_all_neighbors(row, col):
                if self.csp.puzzle[neighbor[0]][neighbor[1]] == 0:
                    if value in self.csp.domains[neighbor]:
                        eliminated_count += 1
            value_constraints.append((value, eliminated_count))
        
        # Sort by ascending constraint count (least constraining first)
        value_constraints.sort(key=lambda x: x[1])
        return [v for v, _ in value_constraints]
    
    def _backtrack(self):
        """Backtracking with MRV and LCV"""
        if self.csp.is_complete():
            return True
        
        # MRV variable selection
        var = self._select_unassigned_variable_mrv()
        if var is None:
            return True
        
        row, col = var
        
        # LCV value ordering
        ordered_values = self._order_domain_values(var)
        
        for value in ordered_values:
            self.assignments += 1
            
            if self.csp.is_consistent(row, col, value):
                self.csp.puzzle[row][col] = value
                saved_domains = deepcopy(self.csp.domains)
                
                # Forward check
                self.csp.domains[(row, col)] = {value}
                if self._forward_check(row, col, value):
                    result = self._backtrack()
                    if result:
                        return True
                
                self.csp.domains = saved_domains
                self.csp.puzzle[row][col] = 0
                self.backtracks += 1
        
        return False
    
    def _forward_check(self, row, col, value):
        """Forward checking"""
        for neighbor in self.csp.get_all_neighbors(row, col):
            ni, nj = neighbor
            if self.csp.puzzle[ni][nj] == 0:
                if value in self.csp.domains[(ni, nj)]:
                    self.csp.domains[(ni, nj)].discard(value)
                    if len(self.csp.domains[(ni, nj)]) == 0:
                        return False
        return True

# Test heuristic solver
print("=== Variable and Value Ordering Heuristics ===\n")

medium_puzzle = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

csp = SudokuCSP(medium_puzzle)
csp.print_puzzle("Medium Difficulty Puzzle")

solver = HeuristicSolver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved with Heuristics")
    print(f"Assignments: {solver.assignments}")
    print(f"Backtracks: {solver.backtracks}")
    print(f"Time: {elapsed:.4f} seconds")
    
    print("\n--- MRV Selection Trace (first 5) ---")
    for i, (cell, domain_size) in enumerate(solver.mrv_selections[:5]):
        print(f"{i+1}. Selected cell {cell} with domain size {domain_size}")

# Compare all approaches
print("\n=== Performance Comparison on Medium Puzzle ===\n")

approaches = {
    'Naive Backtracking': NaiveBacktrackingSolver,
    'Forward Checking': ForwardCheckingSolver,
    'MRV + LCV Heuristics': HeuristicSolver
}

for name, SolverClass in approaches.items():
    csp_test = SudokuCSP(medium_puzzle)
    solver_test = SolverClass(csp_test)
    solved_test, time_test = solver_test.solve()
    print(f"{name}:")
    print(f"  Assignments: {solver_test.assignments}")
    print(f"  Time: {time_test:.4f}s")
    print()

=== Variable and Value Ordering Heuristics ===


Medium Difficulty Puzzle
─────────────────────────
. . . │ 6 . . │ 4 . . 
7 . . │ . . 3 │ 6 . . 
. . . │ . 9 1 │ . 8 . 
─────────────────────────
. . . │ . . . │ . . . 
. 5 . │ 1 8 . │ . . 3 
. . . │ 3 . 6 │ . 4 5 
─────────────────────────
. 4 . │ 2 . . │ . 6 . 
9 . 3 │ . . . │ . . . 
. 2 . │ . . . │ 1 . . 


Solved with Heuristics
─────────────────────────
5 8 1 │ 6 7 2 │ 4 3 9 
7 9 2 │ 8 4 3 │ 6 5 1 
3 6 4 │ 5 9 1 │ 7 8 2 
─────────────────────────
4 3 8 │ 9 5 7 │ 2 1 6 
2 5 6 │ 1 8 4 │ 9 7 3 
1 7 9 │ 3 2 6 │ 8 4 5 
─────────────────────────
8 4 5 │ 2 1 9 │ 3 6 7 
9 1 3 │ 7 6 8 │ 5 2 4 
6 2 7 │ 4 3 5 │ 1 9 8 

Assignments: 440
Backtracks: 382
Time: 0.1720 seconds

--- MRV Selection Trace (first 5) ---
1. Selected cell (1, 4) with domain size 1
2. Selected cell (2, 3) with domain size 1
3. Selected cell (1, 3) with domain size 1
4. Selected cell (0, 5) with domain size 1
5. Selected cell (0, 0) with domain size 1

=== Performance Compa

In [7]:
def benchmark_solvers(puzzles, puzzle_names):
    """Compare all solving techniques on multiple puzzles"""
    
    solvers = {
        'Naive': NaiveBacktrackingSolver,
        'Forward Checking': ForwardCheckingSolver,
        'AC-3': AC3Solver,
        'Heuristics (MRV+LCV)': HeuristicSolver
    }
    
    results = {name: [] for name in solvers}
    
    for puzzle_name, puzzle in zip(puzzle_names, puzzles):
        print(f"\n{'='*60}")
        print(f"Testing: {puzzle_name}")
        print(f"{'='*60}\n")
        
        # Count givens
        givens = sum(1 for row in puzzle for val in row if val != 0)
        empty = 81 - givens
        print(f"Givens: {givens}, Empty cells: {empty}\n")
        
        for solver_name, SolverClass in solvers.items():
            print(f"Running {solver_name}...", end=" ")
            
            csp = SudokuCSP(puzzle)
            solver = SolverClass(csp)
            
            try:
                solved, elapsed = solver.solve()
                
                if solved:
                    print(f"✓ {elapsed:.4f}s, {solver.assignments} assignments")
                    results[solver_name].append({
                        'puzzle': puzzle_name,
                        'time': elapsed,
                        'assignments': solver.assignments,
                        'solved': True
                    })
                else:
                    print("✗ Failed")
                    results[solver_name].append({
                        'puzzle': puzzle_name,
                        'solved': False
                    })
            except Exception as e:
                print(f"✗ Error: {e}")
                results[solver_name].append({
                    'puzzle': puzzle_name,
                    'solved': False
                })
    
    return results

# Define test puzzles of increasing difficulty
easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

medium_puzzle = [
    [0, 0, 0, 0, 0, 0, 6, 8, 0],
    [0, 0, 0, 0, 7, 3, 0, 0, 9],
    [3, 0, 9, 0, 0, 0, 0, 4, 5],
    [4, 9, 0, 0, 0, 0, 0, 0, 0],
    [8, 0, 3, 0, 5, 0, 9, 0, 2],
    [0, 0, 0, 0, 0, 0, 0, 3, 6],
    [9, 6, 0, 0, 0, 0, 3, 0, 8],
    [7, 0, 0, 6, 8, 0, 0, 0, 0],
    [0, 2, 8, 0, 0, 0, 0, 0, 0]
]

hard_puzzle = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

print("=== Comprehensive Performance Benchmark ===")

puzzles = [easy_puzzle, medium_puzzle, hard_puzzle]
names = ["Easy (36 givens)", "Medium (27 givens)", "Hard (21 givens)"]

results = benchmark_solvers(puzzles, names)

# Summary table
print("\n" + "="*60)
print("SUMMARY: Average Assignments by Difficulty")
print("="*60 + "\n")

print(f"{'Solver':<25} {'Easy':<12} {'Medium':<12} {'Hard':<12}")
print("─" * 60)

for solver_name in results:
    row = f"{solver_name:<25}"
    for puzzle_name in names:
        matching = [r for r in results[solver_name] if r['puzzle'] == puzzle_name and r['solved']]
        if matching:
            avg_assigns = matching[0]['assignments']
            row += f"{avg_assigns:<12}"
        else:
            row += f"{'FAILED':<12}"
    print(row)

=== Comprehensive Performance Benchmark ===

Testing: Easy (36 givens)

Givens: 30, Empty cells: 51

Running Naive... ✓ 0.0384s, 37652 assignments
Running Forward Checking... ✓ 0.1931s, 490 assignments
Running AC-3... ✓ 0.0100s, 0 assignments
Running Heuristics (MRV+LCV)... ✓ 0.0325s, 51 assignments

Testing: Medium (27 givens)

Givens: 27, Empty cells: 54

Running Naive... ✓ 0.0683s, 78050 assignments
Running Forward Checking... ✓ 1.7856s, 6401 assignments
Running AC-3... ✓ 0.0168s, 10 assignments
Running Heuristics (MRV+LCV)... ✓ 0.1040s, 310 assignments

Testing: Hard (21 givens)

Givens: 23, Empty cells: 58

Running Naive... ✓ 7.6301s, 7914530 assignments
Running Forward Checking... ✓ 50.6314s, 185660 assignments
Running AC-3... ✓ 0.6382s, 1183 assignments
Running Heuristics (MRV+LCV)... ✓ 0.1392s, 440 assignments

SUMMARY: Average Assignments by Difficulty

Solver                    Easy         Medium       Hard        
────────────────────────────────────────────────────────────

In [8]:
import random

class LocalSearchSolver:
    """Min-conflicts local search for Sudoku"""
    
    def __init__(self, csp, max_iterations=10000):
        self.csp = csp
        self.max_iterations = max_iterations
        self.iterations = 0
        self.conflict_history = []
    
    def solve(self):
        """Attempt to solve using local search"""
        self.iterations = 0
        self.conflict_history = []
        
        start_time = time.time()
        
        # Initialize: fill all empty cells with random valid values
        self._random_initialization()
        
        initial_conflicts = self._count_conflicts()
        print(f"Initial conflicts: {initial_conflicts}\n")
        
        # Iterative improvement
        for i in range(self.max_iterations):
            self.iterations += 1
            
            conflicts = self._count_conflicts()
            self.conflict_history.append(conflicts)
            
            if conflicts == 0:
                elapsed = time.time() - start_time
                print(f"✓ Solution found after {i+1} iterations!")
                return True, elapsed
            
            if i % 100 == 0:
                print(f"Iteration {i}: {conflicts} conflicts")
            
            # Select variable with conflicts
            conflicted = self._get_conflicted_variables()
            if not conflicted:
                break
            
            var = random.choice(conflicted)
            
            # Try to reduce conflicts by changing this variable's value
            self._min_conflicts_move(var)
        
        elapsed = time.time() - start_time
        final_conflicts = self._count_conflicts()
        print(f"\n✗ Failed to find solution")
        print(f"Final conflicts: {final_conflicts}")
        return False, elapsed
    
    def _random_initialization(self):
        """Fill empty cells with random values"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    # Random value from domain
                    self.csp.puzzle[i][j] = random.randint(1, 9)
    
    def _count_conflicts(self):
        """Count total constraint violations"""
        conflicts = 0
        
        # Check rows
        for i in range(self.csp.size):
            row_vals = [self.csp.puzzle[i][j] for j in range(self.csp.size)]
            conflicts += len(row_vals) - len(set(row_vals))
        
        # Check columns
        for j in range(self.csp.size):
            col_vals = [self.csp.puzzle[i][j] for i in range(self.csp.size)]
            conflicts += len(col_vals) - len(set(col_vals))
        
        # Check boxes
        for box_i in range(3):
            for box_j in range(3):
                box_vals = []
                for i in range(box_i * 3, (box_i + 1) * 3):
                    for j in range(box_j * 3, (box_j + 1) * 3):
                        box_vals.append(self.csp.puzzle[i][j])
                conflicts += len(box_vals) - len(set(box_vals))
        
        return conflicts
    
    def _get_conflicted_variables(self):
        """Get list of variables involved in conflicts"""
        conflicted = []
        
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                # Skip given cells
                if (i, j) not in self.csp.domains or len(self.csp.domains[(i, j)]) == 1:
                    if self.csp.puzzle[i][j] != 0 and (i, j) in [(r, c) for r in range(9) for c in range(9) if self.csp.puzzle[r][c] != 0]:
                        continue
                
                # Check if this cell is in conflict
                value = self.csp.puzzle[i][j]
                if self._has_conflict(i, j, value):
                    conflicted.append((i, j))
        
        return conflicted
    
    def _has_conflict(self, row, col, value):
        """Check if this cell/value creates conflicts"""
        # Check row
        for c in range(self.csp.size):
            if c != col and self.csp.puzzle[row][c] == value:
                return True
        
        # Check column
        for r in range(self.csp.size):
            if r != row and self.csp.puzzle[r][col] == value:
                return True
        
        # Check box
        box_row, box_col = (row // 3) * 3, (col // 3) * 3
        for i in range(box_row, box_row + 3):
            for j in range(box_col, box_col + 3):
                if (i, j) != (row, col) and self.csp.puzzle[i][j] == value:
                    return True
        
        return False
    
    def _min_conflicts_move(self, var):
        """Change variable to value with minimum conflicts"""
        row, col = var
        current_value = self.csp.puzzle[row][col]
        
        # Try all values and count conflicts
        min_conflicts = float('inf')
        best_value = current_value
        
        for value in range(1, 10):
            self.csp.puzzle[row][col] = value
            conflicts = self._count_conflicts()
            
            if conflicts < min_conflicts:
                min_conflicts = conflicts
                best_value = value
        
        self.csp.puzzle[row][col] = best_value

# Test local search
print("=== Local Search (Min-Conflicts) ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Attempting with Local Search")

solver = LocalSearchSolver(csp, max_iterations=1000)
solved, elapsed = solver.solve()

print(f"\nTime: {elapsed:.4f}s")
print(f"Iterations: {solver.iterations}")

# Show conflict trajectory
if len(solver.conflict_history) > 10:
    print("\nConflict trajectory (every 100 iterations):")
    for i in range(0, min(len(solver.conflict_history), 1000), 100):
        print(f"  Iteration {i}: {solver.conflict_history[i]} conflicts")

# Compare with systematic search
print("\n--- Comparison: Local Search vs. Backtracking ---\n")

csp_bt = SudokuCSP(easy_puzzle)
solver_bt = HeuristicSolver(csp_bt)
solved_bt, time_bt = solver_bt.solve()

print(f"Local Search: {' Solved' if solved else 'Failed'} in {elapsed:.4f}s, {solver.iterations} iterations")
print(f"Backtracking: {'Solved' if solved_bt else 'Failed'} in {time_bt:.4f}s, {solver_bt.assignments} assignments")
print(f"\nConclusion: Local search struggles with CSPs that require exact solutions!")

=== Local Search (Min-Conflicts) ===


Attempting with Local Search
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 

Initial conflicts: 80

Iteration 0: 80 conflicts
Iteration 100: 28 conflicts
Iteration 200: 19 conflicts
Iteration 300: 19 conflicts
Iteration 400: 19 conflicts
Iteration 500: 19 conflicts
Iteration 600: 19 conflicts
Iteration 700: 19 conflicts
Iteration 800: 19 conflicts
Iteration 900: 19 conflicts

✗ Failed to find solution
Final conflicts: 19

Time: 0.4208s
Iterations: 1000

Conflict trajectory (every 100 iterations):
  Iteration 0: 80 conflicts
  Iteration 100: 28 conflicts
  Iteration 200: 19 conflicts
  Iteration 300: 19 conflicts
  Iteration 400: 19 conflicts
  Iteration 500: 19 conflicts
  Iteration 600: 19 conflicts
  Iteration 7

In [1]:
from z3 import *
import time

def solve_sudoku_z3(puzzle):
    """
    Solve Sudoku using Z3 SMT solver.
    
    Rather than implementing search ourselves, we declare constraints
    and let Z3's sophisticated solver find a satisfying assignment.
    """
    
    # Create 9x9 grid of integer variables
    grid = [[Int(f"cell_{i}_{j}") for j in range(9)] for i in range(9)]
    
    s = Solver()
    
    # Constraint 1: Each cell is between 1 and 9
    print("Adding domain constraints...")
    for i in range(9):
        for j in range(9):
            s.add(And(grid[i][j] >= 1, grid[i][j] <= 9))
    
    # Constraint 2: Rows are distinct (no duplicates)
    print("Adding row constraints...")
    for i in range(9):
        s.add(Distinct(grid[i]))
    
    # Constraint 3: Columns are distinct
    print("Adding column constraints...")
    for j in range(9):
        s.add(Distinct([grid[i][j] for i in range(9)]))
    
    # Constraint 4: 3x3 boxes are distinct
    print("Adding box constraints...")
    for box_i in range(3):
        for box_j in range(3):
            cells = [grid[i][j] 
                     for i in range(box_i*3, box_i*3+3)
                     for j in range(box_j*3, box_j*3+3)]
            s.add(Distinct(cells))
    
    # Constraint 5: Fix given cells
    print("Adding given values...")
    givens = 0
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] != 0:
                s.add(grid[i][j] == puzzle[i][j])
                givens += 1
    
    print(f"Total givens: {givens}")
    print(f"Total constraints added: {len(s.assertions())}")
    print("\nSolving with Z3...")
    
    # Solve
    start = time.time()
    result = s.check()
    elapsed = time.time() - start
    
    if result == sat:
        m = s.model()
        print(f"✓ Solution found in {elapsed:.4f}s")
        
        # Extract solution
        solution = [[0]*9 for _ in range(9)]
        for i in range(9):
            for j in range(9):
                solution[i][j] = m.evaluate(grid[i][j]).as_long()
        
        return solution, elapsed, True
    else:
        print(f"✗ No solution exists")
        return None, elapsed, False

def print_sudoku(puzzle, title="Sudoku"):
    """Pretty print the puzzle"""
    print(f"\n{title}")
    print("─" * 25)
    for i in range(9):
        if i > 0 and i % 3 == 0:
            print("─" * 25)
        row_str = ""
        for j in range(9):
            if j > 0 and j % 3 == 0:
                row_str += "│ "
            val = puzzle[i][j]
            row_str += str(val) if val != 0 else "."
            row_str += " "
        print(row_str)
    print()

# Test on puzzles of varying difficulty
print("=== Z3 SMT Solver for Sudoku ===\n")

# Easy puzzle
easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

print_sudoku(easy_puzzle, "Easy Puzzle (36 givens)")
solution, time_easy, solved = solve_sudoku_z3(easy_puzzle)
if solved:
    print_sudoku(solution, "Solution")

# Hard puzzle
hard_puzzle = [
    [0, 0, 0, 0, 0, 0, 6, 8, 0],
    [0, 0, 0, 0, 7, 3, 0, 0, 9],
    [3, 0, 9, 0, 0, 0, 0, 4, 5],
    [4, 9, 0, 0, 0, 0, 0, 0, 0],
    [8, 0, 3, 0, 5, 0, 9, 0, 2],
    [0, 0, 0, 0, 0, 0, 0, 3, 6],
    [9, 6, 0, 0, 0, 0, 3, 0, 8],
    [7, 0, 0, 6, 8, 0, 0, 0, 0],
    [0, 2, 8, 0, 0, 0, 0, 0, 0]
]

print("\n" + "="*50 + "\n")
print_sudoku(hard_puzzle, "Hard Puzzle (21 givens)")
solution, time_hard, solved = solve_sudoku_z3(hard_puzzle)
if solved:
    print_sudoku(solution, "Solution")

# Analysis
print("\n=== Z3 Performance Summary ===\n")
print(f"Easy puzzle: {time_easy:.4f}s")
print(f"Hard puzzle: {time_hard:.4f}s")
print(f"Ratio: {time_hard/time_easy:.2f}x")

print("\n--- Key Observations ---\n")
print("1. Z3 handles both puzzles efficiently despite difficulty difference")
print("2. Declarative specification: we state constraints, not search strategy")
print("3. Z3 uses sophisticated techniques (clause learning, backjumping)")
print("4. Built-in Distinct() constraint is optimized for all-different")
print("5. No explicit variable ordering or value selection needed")

=== Z3 SMT Solver for Sudoku ===


Easy Puzzle (36 givens)
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 

Adding domain constraints...
Adding row constraints...
Adding column constraints...
Adding box constraints...
Adding given values...
Total givens: 30
Total constraints added: 138

Solving with Z3...
✓ Solution found in 0.0232s

Solution
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 




Hard Puzzle (21 givens)
─────────────────────────
. . . │ . . . │ 6 8 . 
. . . │ . 7 3 │ . . 9 
3 . 9 │ . . . │ . 4 5 
──────────